In [4]:
import arcpy
import pandas as pd
import os
import csv
import subprocess
from time import sleep


In [5]:

def set_years(cf):
    lu_tables = r"M:\code\landuse\lookup_tables"
    lc_dates = f"{lu_tables}/landcover_dates.csv" # T1 and T2 by cofips

    # read in years for county
    dates = pd.read_csv(lc_dates)
    dates = dates.set_index('co_fips')
    global T1_yr
    T1_yr = int(dates.loc[cf, 'T1'])
    global T2_yr
    T2_yr = int(dates.loc[cf, 'T2'])
    del dates
    return T1_yr, T2_yr

def MakeIterDir(parentDir, iterDir):
    """
    make a new folder in parentDir with the same name, indicated by iterDir, and the next iteration number
    """

    tnums = []
    for dir in os.listdir(parentDir):
        if iterDir in dir and len(dir) < 7:
            tnums.append(int(dir.replace(iterDir,"")))

    latestBN = f"{iterDir}{max(tnums)}"
    latestDir = os.path.join(parentDir,latestBN)
    latestDirNFiles = len([name for name in os.listdir(latestDir) if os.path.isfile(os.path.join(latestDir, name))])

    
    if latestDirNFiles == 0:
        # if the latest iteration directory is empty, set at iteration output directory
        LatestIterationDirectory = latestDir
        return LatestIterationDirectory
    else:
        #Otherwise create a new iteration directory
        NewIterDir = os.path.join(parentDir,f"{iterDir}{max(tnums)+1}")
        os.mkdir(NewIterDir)
        return NewIterDir

In [6]:
hex_ac = '100ac'
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps(f'Map1000ac')[0] #mapframe is Map1000ac for both 1000 and 100 acre hexes
LO = aprx.listLayouts('Layout2')[0]
lyt = aprx.listLayouts('Layout2')[0]

water_poly = r"M:\projects\data_processing\data_processing.gdb/bay_and_ocean"
bay_poly = "M:\data\BayEstuary\Chesapeake_Bay_Alb.shp"
if hex_ac == "100ac":
    symb_lyr = r"M:\projects\tc_fact_sheet\symbology\100m_hexsym_230224.lyrx"
if hex_ac == "1000ac":
    symb_lyr = r"M:\projects\tc_fact_sheet\symbology\hexsym_1ac_updated_colors_230224.lyrx"
bound_symb = r"M:\projects\tc_fact_sheet\symbology\1pt_hollow_70pctBLK_bound.lyrx"
boundPath = r"J:\GIS\CBP_Obj_1\data\county_sets\tl_2017_bay_county.shp"


#hide all layers
for lyr in map.listLayers():
    lyr.visible = False

base_lyr = map.listLayers("Light Gray Base")[0]
base_lyr.visible = True

In [9]:
lcdates_csv = r"M:\projects\tc_fact_sheet\data\county_list_legendloc.csv"
codf = pd.read_csv(lcdates_csv)
#pandas upgrade required to use .query, PROPY env does not like that version 
omit = ['wash_11001', 'bedf_51515']
# codf = codf[~codf['cofips'].isin(omit)]
# codf = codf[codf.Size.isin(['s','xs'])]
# codf = codf[codf['cofips'].isin(['math_51115'])]
len(codf)
                  
pngDIR = "M:/projects/tc_fact_sheet/map_exports/auto"
IterationDir = MakeIterDir(pngDIR, "test")

fail_list = []
counter = 0
# small_counties = codf[codf.Size.isin(['s','xs'])] # this is now handed by the hex_ac variable
for index, row in codf.iterrows():
    counter += 1
#     if counter > 3:
#         break
    for ext_hex in map.listLayers(f"*_hex_tajoined3"):
        map.removeLayer(ext_hex)
    cf = row['cofips']
    T1 = row['T1']
    T2 = row['T2']
    QA = row['QA']
    LegendLoc = row['Legend']
    size = row['Size']
    if size in ['s','xs']:
        hex_ac = "100ac"
        hex_query = r"all_lu > ( 404800/ 2) And wat_lu < (all_lu* 0.9)"
        hexta = f"M:/projects/tc_fact_sheet/data/prod/{cf}_100ac/{cf}_hex_tajoined3.shp"
    else:
        hex_ac = "1000ac"
        hex_query = r"all_lu > ( 4046000/ 2) And wat_lu < (all_lu* 0.9)"
        hexta = f"M:/projects/tc_fact_sheet/data/prod/{cf}/{cf}_hex_tajoined3.shp"
        
    FIPS = cf.split('_')[1]

    print(f"{cf} {counter}/{len(codf)}", end='\r')

    FIPS = cf.split('_')[1]
        
    try:

        map.addDataFromPath(hexta)
        hex_lyr = map.listLayers(f"{cf}_hex_tajoined3")[0] # FIND HEX LAYER IF IT EXISTS
        hex_lyr.visible = True
        desc = arcpy.Describe(hex_lyr)
        flds = desc.fields


        arcpy.SelectLayerByAttribute_management(hex_lyr, "CLEAR_SELECTION")
        hex_lyr.definitionQuery = hex_query
    except:
        msg = f"{cf} hex_lyr does not exist in {map.name}"
        print(msg)
        fail_list.append(msg)
        pass
    try:
        map.addDataFromPath(boundPath)
        bound_lyr = map.listLayers(f"tl_2017_bay_county")[0]
        arcpy.management.ApplySymbologyFromLayer(bound_lyr, bound_symb)
        bound_lyr.definitionQuery = f"GEOID = '{FIPS}'"

    except:
        msg2 = f"{cf} does not exist in bound_lyr"
        print(msg2)
        fail_list.append(msg2)
        hex_lyr.visible = False
        pass

    try:
        arcpy.management.ApplySymbologyFromLayer(hex_lyr, symb_lyr, "VALUE_FIELD tc_chg tc_chg", "MAINTAIN")
    except:
        print(cf)
        msg = f'{cf} hex_lyr failed symbolize'
        fail_list.append(msg)

    mf = lyt.listElements("MAPFRAME_ELEMENT")[0]
    # IF PICTURES ARE RE-ORDERED IN LAYOUT CONTENTS THIS WILL GET CONFUSED.


    mf.camera.setExtent(mf.getLayerExtent(bound_lyr, False, True))
    old_scale = mf.camera.scale
    
    if hex_ac == '1000ac':
        if LegendLoc == 'r':
            cur_legend = 'Legend_BR'
        elif LegendLoc == 'l':
            cur_legend = 'Legend_BL'
        elif LegendLoc == 'b':
            mf.camera.scale = old_scale*1.2
            mf.camera.Y += -4000
            cur_legend = 'Legend_BM'
            
    if hex_ac == '100ac':
        if LegendLoc == 'r':
            cur_legend = 'Legend_BR_tenth_ac'
        if LegendLoc == 'l':
            cur_legend = 'Legend_BL_tenth_ac'
        if LegendLoc == 'b':
            mf.camera.scale = old_scale*1.1
            mf.camera.Y += -500
            cur_legend = 'Legend_BM_tenth_ac'
    
    legend_list = lyt.listElements('PICTURE_ELEMENT')
    for legend in legend_list:
        if legend.name == cur_legend:
            legend.visible = True
        else:
            legend.visible = False

    lyt.exportToPNG(f"{IterationDir}/{cf}_hexmap",resolution=300)

    map.removeLayer(hex_lyr)
    map.removeLayer(bound_lyr)

print('          ')
print(fail_list)
subprocess.Popen(f'explorer /select,{IterationDir}')
print(f"Complete! Outputs are located at... {IterationDir}")

bedf_51515 hex_lyr does not exist in Map1000ac
bedf_51515
wash_11001 hex_lyr does not exist in Map1000ac
wash_11001
lanc_51103 hex_lyr does not exist in Map1000ac
lanc_51103
           206/206
['bedf_51515 hex_lyr does not exist in Map1000ac', 'bedf_51515 hex_lyr failed symbolize', 'wash_11001 hex_lyr does not exist in Map1000ac', 'wash_11001 hex_lyr failed symbolize', 'lanc_51103 hex_lyr does not exist in Map1000ac', 'lanc_51103 hex_lyr failed symbolize']
Complete! Outputs are located at... M:/projects/tc_fact_sheet/map_exports/auto\test33


In [2]:
xcf = "alex_51510"
x_hexta = f"M:/projects/tc_fact_sheet/data/prod/{xcf}_100ac/{xcf}_hex_tajoined3.shp"
map.addDataFromPath(x_hexta)
x_lyr = map.listLayers(f"{xcf}_hex_tajoined3")[0]
print(x_lyr.name)
x_lyr.definitionQuery = "all_lu > ( 4046000/ 2) And wat_lu < (all_lu* 0.9)"
mf.camera.setExtent(mf.getLayerExtent(x_lyr, False, True))

AttributeError: type object 'map' has no attribute 'addDataFromPath'

In [18]:
hex_query = "all_lu > ( 404800/ 2) And wat_lu < (all_lu* 0.9)"
x_lyr.definitionQuery = hex_query


True

In [13]:
hex_ac = '100ac'
LegendLoc = 'r'

if hex_ac == '1000ac':
    if LegendLoc == 'r':
        cur_legend = 'Legend_BR'
    elif LegendLoc == 'l':
        cur_legend = 'Legend_BL'
    elif LegendLoc == 'b':
        mf.camera.scale = old_scale*1.2
        mf.camera.Y += -4000
        cur_legend = 'Legend_BM'

if hex_ac == '100ac':
    if LegendLoc == 'r':
        cur_legend = 'Legend_BR_tenth_ac'
    if LegendLoc == 'l':
        cur_legend = 'Legend_BL_tenth_ac'
    if LegendLoc == 'b':
        mf.camera.scale = old_scale*1.1
        mf.camera.Y += -500
        cur_legend = 'Legend_BM_tenth_ac'

legend_list = lyt.listElements('PICTURE_ELEMENT')
for legend in legend_list:
    if legend.name == cur_legend:
        legend.visible = True
    else:
        legend.visible = False

In [12]:
mf = lyt.listElements("MAPFRAME_ELEMENT")[0]

'Legend_BM_tenth_ac'
'Legend_BL_tenth_ac'
'Legend_BR_tenth_ac'
'Legend_BM'
'Legend_BL'
'Legend_BR'


In [54]:

legend_list = lyt.listElements('PICTURE_ELEMENT')
for legend in legend_list:
    if legend.name == cur_legend:
        legend.visible = True
    else:
        legend.visible = False